# add frames to training set
After training an initial network, watch videos with overlaid tracking and dentify frames that should be re-included in the training set (with annotations initialized by current network's output). Use this notebook to merge these frames with an existing dataset. 

In [7]:
from deepposekit.io.utils import merge_new_images
from deepposekit.io import VideoReader
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import cv2
import os
sys.path.append(os.path.abspath(os.path.join('..')))  # add partent directory
from predict.utils import make_tracking_video
%load_ext autoreload
%autoreload 2

# settings
old_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_9.h5'
merged_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_temp.h5'
video = r'Z:\C1\free\vids\20201001_Fred\concatenated.avi'

frame_nums = [0, 1000, 2000]
frame_nums.sort()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# load predictions
# (assumes video_tracking.csv already exists)
predictions = pd.read_csv(os.path.splitext(video)[0] + '_tracking.csv').to_numpy()[:,1:]
predictions = predictions.reshape((-1, int(predictions.shape[1]/3), 3))  # (frane_num X feature_num X (x,y,confidence))
keypoints = predictions[frame_nums]

I load images by scrolling through all the frames below... Directly indexing into the video results in incorrect frames being drawn from the original video. This is probably because the original (but not the tracking) video has an incorrect frame count number. I coudln't figure out how to fix this in opencv...

In [9]:
# load images
reader = VideoReader(video, batch_size=1, fast_frame_count=True)

images = []
for i in tqdm(range(frame_nums[-1]+1)):
    if i in frame_nums:
        images.append(reader.read())
    else:
        reader.read()
images = np.stack([i[...,0][...,np.newaxis] for i in images])  # replace color with singleton dimension and concatenate

# images = np.concatenate([reader[idx][...,0] for idx in frame_nums])[...,np.newaxis]  # direct indexing into the reader draws incorrect frames for mysterious reasons...

100%|█████████████████████████████████████████████████████████████████████████| 2001/2001 [00:00<00:00, 3598.10it/s]


In [5]:
# make merged dataset
merge_new_images(
    datapath=old_dataset,
    merged_datapath=merged_dataset,
    images=images,
    frame_numbers=frame_numbers,
    video_names=video_names,
    keypoints=keypoints,
    overwrite=False
)


# add frames from spreadsheet
Download [this](https://docs.google.com/spreadsheets/d/1_kMZSCiMDglHKb363tWUf2j9IzwLFfPrArS-zXin_94/edit?usp=sharing) spreadsheet as a .csv, and add frames and videos to training as specified in the spreadsheet. *Only works for head free videos.*

In [1]:
from deepposekit.io.utils import merge_new_images
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import h5py
import yaml
import sys
import cv2
import os
sys.path.append(os.path.abspath(os.path.join('..')))  # add partent directory
root_dir = r'D:\github\fish-tracking'  # github repo
%load_ext autoreload
%autoreload 2


# settings
old_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_13.h5'
merged_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_14.h5'
spreadsheet = r'D:\github\fish-tracking\label\c1_tracking_errors.csv'


# inits
with open(os.path.join(root_dir, 'config.yaml'), 'r') as file:
    cfg = yaml.safe_load(file)['free']

tracking_errors = pd.read_csv(spreadsheet)
vidnames = [os.path.join(cfg['data_dir'], str(r['Date']) + '_' + r['Fish'], 'concatenated.avi')
            for (i, r) in tracking_errors.iterrows()]
tracking_errors['FileName'] = vidnames
vidnames = np.unique(vidnames)

tracking_errors

,Date,Fish,Frame,View(s),Error,User,Notes,FileName
0,20201110,Joao,1392,NaN,NaN,NaN,NaN,Z:\C1\free\vids\20201110_Joao\concatenated.avi
1,20201110,Joao,8025,NaN,NaN,NaN,NaN,Z:\C1\free\vids\20201110_Joao\concatenated.avi
2,20201110,Joao,76630,NaN,NaN,NaN,NaN,Z:\C1\free\vids\20201110_Joao\concatenated.avi
3,20201110,Joao,157961,NaN,NaN,NaN,NaN,Z:\C1\free\vids\20201110_Joao\concatenated.avi


In [2]:
# get frames
images, keypoints, frame_numbers, video_names = [], [], [], []
for idx, vidname in enumerate(vidnames):
    predictionsname = os.path.splitext(vidname)[0] + '_tracking.csv'
    
    if os.path.exists(vidname) and os.path.exists(predictionsname):
        vid_bins = tracking_errors['FileName']==vidname
        frames = tracking_errors.loc[vid_bins, 'Frame'].tolist()
        frames = np.sort(np.array(frames))
        frame_numbers += list(frames)
        video_names += [vidname]*len(frames)
        
        print('\n({:3}/{}) {} frames:\n{}'.format(
            idx+1, len(vidnames), vidname, frames))

        # load predictions
        predictions = pd.read_csv(predictionsname, nrows=(frames[-1]+1)).to_numpy()[:,1:]
        predictions = predictions.reshape((-1, int(predictions.shape[1]/3), 3))  # (frame_num X feature_num X (x,y,confidence))
        keypoints.append(predictions[frames])
        del predictions

        # load frame
        reader = cv2.VideoCapture(vidname)
        for i in range(frames[-1]+1):
            if np.isin(i, frames):
                images.append(reader.read()[1])
            else:        
                reader.read()
        reader.release()
        
    else:
        print('{}: video does not exist!'.format(vidname))

images = np.stack([i[...,0][...,np.newaxis] for i in images])  # replace color with singleton dimension and concatenate
keypoints = np.concatenate(keypoints, axis=0)


(  1/1) Z:\C1\free\vids\20201110_Joao\concatenated.avi frames:
[  1392   8025  76630 157961]


In [6]:
# make merged dataset
merge_new_images(
    datapath=old_dataset,
    merged_datapath=merged_dataset,
    images=images,
    frame_numbers=frame_numbers,
    video_names=video_names,
    keypoints=keypoints,
    overwrite=False
)

# check new dataset
with h5py.File(merged_dataset, 'r') as f:
    temp = {key: value[...] for key, value in f.items()}

for n, f in zip(temp['frame_number'], temp['video_name']):
    print('frame {}, video {}'.format(n, f))

frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
frame -1, video b'unknown'
f